In [1]:
import pandas as pd
import numpy as np

In [2]:
austin_outcomes = pd.read_csv("Data/Austin_Animal_Center_Outcomes.csv")
austin_intakes = pd.read_csv("Data/Austin_Animal_Center_Intakes.csv")

In [3]:
bloomington = pd.read_csv("Data/Bloomington_Animal_Shelter_Animals.csv")
louisville = pd.read_csv("Data/Louisville_Animal_IO_Data_5.csv")

In [4]:
dallas14 = pd.read_csv("Data/Dallas_Animal_Shelter_Data_Fiscal_Year_2014.csv", dtype={'Tag Type': 'string', 'Activity Number': 'string', "Service Request Number":'string'})
dallas15 = pd.read_csv("Data/Dallas_Animal_Shelter_Data_Fiscal_Year_2015.csv", dtype={'Tag Type': 'string', 'Activity Number': 'string', "Service Request Number":'string'})
dallas16 = pd.read_csv("Data/Dallas_Animal_Shelter_Data_Fiscal_Year_2016.csv", dtype={'Tag Type': 'string', 'Activity Number': 'string', "Service Request Number":'string'})
dallas17 = pd.read_csv("Data/Dallas_Animal_Shelter_Data_Fiscal_Year_2017.csv", dtype={'Tag Type': 'string', 'Activity Number': 'string', "Service Request Number":'string'})
dallas18 = pd.read_csv("Data/Dallas_Animal_Shelter_Data_Fiscal_Year_2018.csv", dtype={'Tag Type': 'string', 'Activity Number': 'string', "Service Request Number":'string'})
dallas19 = pd.read_csv("Data/Dallas_Animal_Shelter_Data_Fiscal_Year_2019.csv", dtype={'Tag Type': 'string', 'Activity Number': 'string', "Service Request Number":'string'})
dallas20 = pd.read_csv("Data/Dallas_Animal_Shelter_Data_Fiscal_Year_2020.csv", dtype={'Tag Type': 'string', 'Activity Number': 'string', "Service Request Number":'string'})

## Ongoing Data Wrangling To Do List

- remove animals that aren't cats and dogs
- remove all rows without outcomes
- standardize column names across locations
- add location columns for each dataframe
- make sure animal IDs don't overlap with others, might need to merge it with their location for unique IDs
- format Dates, Times, length of stay, age across each dataset
- merge Dallas together into one dataframe
- plot years of data to determine if there's enough data from the first recorded years in each location
- standardize categorical intake and outcomes entries across each location

### Austin Dataframe Formatting

In [5]:
austin_intakes.drop(['Name', 'MonthYear', 'Found Location'],axis=1, inplace=True)

austin_intakes.rename(columns={'Animal ID':'animal_id',
                      'Animal Type':'animal_type',
                      'DateTime':'intake_date',
                      'Intake Type':'intake_type',
                      'Intake Condition':'intake_condition',
                      'Breed':'breed',
                      'Color':'color',
                      'Sex upon Intake':'intake_sex',
                      'Age upon Intake':'intake_age'}, inplace=True)

In [6]:
austin_outcomes.drop(['Name', 'MonthYear'],axis=1, inplace=True)
austin_outcomes.rename(columns={'Animal ID':'animal_id',
                      'Animal Type':'animal_type',
                      'DateTime':'outcome_date',
                      'Outcome Type':'outcome_type',
                      'Outcome Subtype':'outcome_subtype',
                      'Breed':'breed',
                      'Color':'color',
                      'Sex upon Outcome':'outcome_sex',
                      'Age upon Outcome':'outcome_age',
                      'Date of Birth':'birth_date'}, inplace=True)

In [7]:
austin = austin_intakes.merge(austin_outcomes, how='left')
austin.head()

,animal_id,intake_date,intake_type,intake_condition,animal_type,intake_sex,intake_age,breed,color,outcome_date,birth_date,outcome_type,outcome_subtype,outcome_sex,outcome_age
0,A786884,01/03/2019 04:19:00 PM,Stray,Normal,Dog,Neutered Male,2 years,Beagle Mix,Tricolor,01/08/2019 03:11:00 PM,01/03/2017,Transfer,Partner,Neutered Male,2 years
1,A706918,07/05/2015 12:59:00 PM,Stray,Normal,Dog,Spayed Female,8 years,English Springer Spaniel,White/Liver,07/05/2015 03:13:00 PM,07/05/2007,Return to Owner,NaN,Spayed Female,8 years
2,A724273,04/14/2016 06:43:00 PM,Stray,Normal,Dog,Intact Male,11 months,Basenji Mix,Sable/White,04/21/2016 05:17:00 PM,04/17/2015,Return to Owner,NaN,Neutered Male,1 year
3,A665644,10/21/2013 07:59:00 AM,Stray,Sick,Cat,Intact Female,4 weeks,Domestic Shorthair Mix,Calico,10/21/2013 11:39:00 AM,09/21/2013,Transfer,Partner,Intact Female,4 weeks
4,A682524,06/29/2014 10:38:00 AM,Stray,Normal,Dog,Neutered Male,4 years,Doberman Pinsch/Australian Cattle Dog,Tan/Gray,07/02/2014 02:16:00 PM,06/29/2010,Return to Owner,NaN,Neutered Male,4 years


### Louisville Dataframe Formatting

In [8]:
louisville.drop(['SecondaryColor','IntakeReason','IntakeInternalStatus', 'OutcomeReason','OutcomeInternalStatus'], axis=1, inplace=True)

In [9]:
louisville.rename(columns={'AnimalID':'animal_id',
                          'AnimalType':'animal_type',
                          'IntakeDate':'intake_date',
                          'IntakeType':'intake_type',
                          'IntakeSubtype':'intake_subtype',
                          'PrimaryColor':'color',
                          'PrimaryBreed':'breed',
                          'SecondaryBreed':'breed_2',
                          'Gender':'gender',
                          'DOB':'birth_date',
                          'IntakeAsilomarStatus':'intake_condition',
                          'ReproductiveStatusAtIntake':'intake_repro',
                          'OutcomeDate':'outcome_date',
                          'OutcomeType':'outcome_type',
                          'OutcomeSubtype':'outcome_subtype',
                          'OutcomeAsilomarStatus':'outcome_condition',
                          'ReproductiveStatusAtOutcome':'outcome_repro',
                          }, inplace=True)
louisville.head()

,animal_id,animal_type,intake_date,intake_type,intake_subtype,color,breed,breed_2,gender,birth_date,intake_condition,intake_repro,outcome_date,outcome_type,outcome_subtype,outcome_condition,outcome_repro
0,A366370,CAT,2008-11-07 10:50:00,STRAY,OTC,WHITE,DOMESTIC SHORTHAIR,NaN,NEUTERED MALE,NaN,HEALTHY,ALTERED,2008-11-12 15:46:00,EUTH,FERAL,UNHEALTHY/UNTREATABLE,ALTERED
1,A366531,CAT,2008-11-10 10:20:00,STRAY,OTC,BLACK,DOMESTIC SHORTHAIR,DOMESTIC SHORTHAIR,UNKNOWN,NaN,HEALTHY,UNKNOWN,2008-11-19 20:10:00,EUTH,CONTAG DIS,HEALTHY,UNKNOWN
2,A532367,BIRD,2014-07-23 23:21:00,CONFISCATE,CRUELTY,RED,CHICKEN,NaN,MALE,NaN,HEALTHY,FERTILE,2014-11-05 15:49:00,TRANSFER,NaN,HEALTHY,FERTILE
3,A532474,OTHER,2014-07-24 18:29:00,ET REQUEST,NaN,BROWN,BAT,NaN,UNKNOWN,NaN,HEALTHY,UNKNOWN,2014-07-24 23:59:00,EUTH,MEDICAL,HEALTHY,UNKNOWN
4,A281756,DOG,2006-09-11 18:10:00,OWNER SUR,OTC,WHITE,PIT BULL TERRIER,NaN,MALE,2005-09-11 00:00:00,HEALTHY,FERTILE,2006-09-12 13:44:00,EUTH,TIME/SPACE,HEALTHY,FERTILE


### Dallas Dataframe Formatting

In [10]:
dallas1718 = dallas17.append(dallas18)
dallas1718.rename(columns={'Animal_Id':'Animal Id',
                           'Animal_Type':'Animal Type',
                           'Animal_Breed':'Animal Breed',
                           'Kennel_Number':'Kennel Number',
                           'Kennel_Status':'Kennel Status',
                           'Tag_Type':'Tag Type',
                           'Activity_Number':'Activity Number',
                           'Activity_Sequence':'Activity Sequence',
                           'Source_Id':'Source Id',
                           'Census_Tract':'Census Tract',
                           'Council_District':'Council District',
                           'Intake_Type':'Intake Type',
                           'Intake_Subtype':'Intake Subtype',
                           'Intake_Total':'Intake Total',
                           'Staff_Id':'Staff Id',
                           'Intake_Date':'Intake Date',
                           'Intake_Time':'Intake Time',
                           'Due_Out':'Due Out',
                           'Intake_Condition':'Intake Condition',
                           'Hold_Request':'Hold Request',
                           'Outcome_Type':'Outcome Type',
                           'Outcome_Subtype':'Outcome Subtype',
                           'Outcome_Date':'Outcome Date',
                           'Outcome_Time':'Outcome Time',
                           'Receipt_Number':'Receipt Number',
                           'Impound_Number':'Impound Number',
                           'Service_Request_Number':'Service Request Number',
                           'Outcome_Condition':'Outcome Condition',
                           'Chip_Status':'Chip Status',
                           'Animal_Origin':'Animal Origin',
                           'Additional_Information':'Additional Information'
                          }, inplace=True)

dallas16.rename(columns={
                        'Animal ID':'Animal Id',
                        'Staff ID':'Staff Id',
                        'Source ID':'Source Id'
                        }, inplace=True)

In [11]:
dallas = dallas14.append([dallas15,dallas16,dallas1718, dallas19, dallas20])
dallas.shape

(236972, 34)

In [12]:
dallas.drop(['Kennel Number','Kennel Status','Tag Type',
             'Activity Number','Activity Sequence','Source Id',
             'Census Tract', 'Council District', 'Intake Total', 'Reason', 'Staff Id',
            'Due Out', 'Hold Request','Receipt Number', 'Impound Number',
            'Service Request Number', 'Chip Status', 'Animal Origin',
            'Additional Information'], axis=1, inplace=True)

In [13]:
dallas.rename(columns={'Animal Id':'animal_id',
                       'Animal Type':'animal_type',
                       'Animal Breed':'breed',
                       'Intake Type':'intake_type',
                       'Intake Subtype':'intake_subtype',
                       'Intake Date':'intake_date',
                       'Intake Time':'intake_time',
                       'Intake Condition':'intake_condition',
                       'Outcome Type':'outcome_type',
                       'Outcome Date':'outcome_date',
                       'Outcome Time':'outcome_time',
                       'Outcome Condition':'outcome_condition',
                       'Outcome Subtype':'outcome_subtype',
                       'Month':'month',
                       'Year':'year'
    
}, inplace=True)
dallas.head()

,animal_id,animal_type,breed,intake_type,intake_subtype,intake_date,intake_time,intake_condition,outcome_type,outcome_date,outcome_time,outcome_condition,month,year,outcome_subtype
0,A0000575,CAT,DOMESTIC SH,STRAY,CONFINED,10/02/2014 12:00:00 AM,12/31/1899 11:56:00 AM,TREATABLE REHABILITABLE NON-CONTAGIOUS,ADOPTION,10/12/2014 12:00:00 AM,12/31/1899 03:25:00 PM,TREATABLE REHABILITABLE NON-CONTAGIOUS,OCT.2014,FY2015,NaN
1,A0008962,DOG,LABRADOR RETR,CONFISCATED,KEEP SAFE,09/24/2015 12:00:00 AM,12/31/1899 03:50:00 PM,TREATABLE REHABILITABLE NON-CONTAGIOUS,EUTHANIZED,10/04/2015 12:00:00 AM,12/31/1899 12:22:00 PM,TREATABLE MANAGEABLE NON-CONTAGIOUS,SEP.2015,FY2015,NaN
2,A0121376,DOG,GERM SHEPHERD,STRAY,CONFINED,05/01/2015 12:00:00 AM,12/31/1899 12:09:00 PM,TREATABLE MANAGEABLE NON-CONTAGIOUS,EUTHANIZED,05/03/2015 12:00:00 AM,12/31/1899 11:53:00 AM,TREATABLE MANAGEABLE NON-CONTAGIOUS,MAY.2015,FY2015,NaN
3,A0129114,CAT,DOMESTIC SH,OWNER SURRENDER,GENERAL,09/19/2015 12:00:00 AM,12/31/1899 04:46:00 PM,TREATABLE REHABILITABLE NON-CONTAGIOUS,ADOPTION,10/26/2015 12:00:00 AM,12/31/1899 02:09:00 PM,TREATABLE REHABILITABLE NON-CONTAGIOUS,SEP.2015,FY2015,NaN
4,A0157434,DOG,ROTTWEILER,OWNER SURRENDER,- DEAD ON ARRIVAL,12/03/2014 12:00:00 AM,12/31/1899 08:06:00 PM,UNHEALTHY UNTREATABLE NON-CONTAGIOUS,DEAD ON ARRIVAL,12/04/2014 12:00:00 AM,12/31/1899 12:00:00 PM,UNHEALTHY UNTREATABLE NON-CONTAGIOUS,DEC.2014,FY2015,NaN
